##### Translation using facebook NLLB 200 distilled model 

Using model loading for easier definition of source and output languages.

Using as texts to translate the review titles from preprocessed dataset.

In [ ]:
# imports

In [ ]:
# Load dataset and convert to dataframe

In [ ]:
# Define the dictionary mapping languages to language codes
language_code_dict = {
    'en': 'eng_Latn',
    'it': 'ita_Latn',
    'es': 'spa_Latn',
    'fr': 'fra_Latn',
    'de': 'deu_Latn',
    'ja': 'jpn_Japn'
}

In [ ]:
def read_texts_and_language_codes(df):
    # Initialize lists to store texts and language codes
    texts = []
    language_codes = []
    
    # Iterate over rows of the dataframe
    for index, row in df.iterrows():
        # Append text to the list
        texts.append(row['review_title'])
        
        # Map language code to language code from dictionary and append to the list
        language_codes.append(language_code_dict[row['language']])
    
    return texts, language_codes


In [ ]:
texts, language_codes = read_texts_and_language_codes(df)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Definition of general variables for all
# Define the model checkpoint
model_checkpoint = "facebook/nllb-200-distilled-600M"

# Initialize model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, token=True)

# Initialize lists to store translated texts
translated_texts = []

# Iterate through the texts list
i = 0 # Temporal iterator for accessing languages_codes list
for text in texts:
    # Define source language from language codes list
    src_lang = str(language_codes[i])

    # Initialize tokenizer for input language
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, token=True, src_lang=src_lang)

    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt")
    
    # Generate translation
    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=30
    )
    
    # Decode translated tokens
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    
    # Append translated text to the list
    translated_texts.append(translated_text)

# Print translated texts
for original, translated in zip(texts, translated_texts):
    print("Original Text:", original)
    print("Translated Text:", translated)
    print()
